In [1]:
import pandas as pd
import numpy as np

data=pd.read_excel("Extracted_data.xlsx")

data = data.rename(columns={'Number_of_Years_in_Business': 'NUMBER_OF_YEARS_IN_BUSINESS', 'Inventory': 'INVENTORY', 'Earnings_Before_Tax' :'EARNING_BEFORE_TAX', 
                           'Current_Assets':'CURRENT_ASSETS', 'Short_Term_Liabilities':'Short_term_Liabilities', 'Finance_Cost':'Finance_cost', 
                           'Long_Term_Liabilities':'Long_term_Liabilities'})

len(data)

4206

In [2]:
def Credit_Score(df, sector):
    
    #LIQUIDITY RELATIVE TO SECTOR BEST
    if sector < 6 or sector==7:
        LIQUIDITY_RELATIVE_TO_SECTOR_BEST = -0.87326
    elif sector==6:
        LIQUIDITY_RELATIVE_TO_SECTOR_BEST = -1.5017   
    else:
        LIQUIDITY_RELATIVE_TO_SECTOR_BEST = -0.80308
    
    company_age=-0.035031731*df.NUMBER_OF_YEARS_IN_BUSINESS #max year 23
    Com_adj=np.where(company_age < -0.7706981, -0.7706981, company_age)
    Best_values= -2.695374908-1.10074827-1.68564152+LIQUIDITY_RELATIVE_TO_SECTOR_BEST
    
    #sector growth
    if sector==1 or sector==7:
        sector_growth=0.0278499
    elif sector>1 and sector <6:
        sector_growth=-0.09312945
    elif sector ==6:
        sector_growth=-0.0989015
    else:
        sector_growth =-0.29093514
    
    #sector evaluation   
    if sector in [1, 7, 9, 17]:
        sector_evaluation=0
    elif sector== 6 or sector ==12:
        sector_evaluation=0.7600768
    else:
        sector_evaluation=0.31864218
        
    #Best Score
    Credit_sc_adj_b=0.332094193 + sector_growth + sector_evaluation + Com_adj + Best_values
    best_score=(3.838634311-Credit_sc_adj_b)/(3.838634311-(-7.868071879))*100

    FC_SHARE = best_score - (best_score*((Best_values+2.695374908)/Best_values))
    ROA_SHARE = best_score - (best_score*((Best_values+1.10074827)/Best_values))
    ATR_SHARE = best_score - (best_score*((Best_values+1.68564152)/Best_values))
    LRS_SHARE = best_score - (best_score*((Best_values-LIQUIDITY_RELATIVE_TO_SECTOR_BEST)/Best_values))
    SOL_SHARE = best_score - (best_score*((Best_values-2.322368)/Best_values))
    
    #LIQUIDITY adjasted
    if sector < 6 or sector==7:
        LIQUIDITY_RELATIVE_TO_SECTOR = ((df.CURRENT_ASSETS-df.INVENTORY)/df.Short_term_Liabilities)*-0.459609287
        liq=np.where(LIQUIDITY_RELATIVE_TO_SECTOR<-0.87326, -0.87326, LIQUIDITY_RELATIVE_TO_SECTOR)
        LIQ_adj=np.where(LIQUIDITY_RELATIVE_TO_SECTOR>0, 0, liq)
    
    elif sector==6:
        LIQUIDITY_RELATIVE_TO_SECTOR = (((df.CURRENT_ASSETS-df.INVENTORY)/df.Short_term_Liabilities)-0.59)*-1.45795761
        liq=np.where(LIQUIDITY_RELATIVE_TO_SECTOR<-1.50169603, -1.50169603, LIQUIDITY_RELATIVE_TO_SECTOR)
        LIQ_adj=np.where(LIQUIDITY_RELATIVE_TO_SECTOR > 0, 0, liq)
    
    else:
        LIQUIDITY_RELATIVE_TO_SECTOR = (((df.CURRENT_ASSETS-df.INVENTORY)/df.Short_term_Liabilities)-0.85)*-1.460143528
        liq=np.where(LIQUIDITY_RELATIVE_TO_SECTOR< -0.80308, -0.80308, LIQUIDITY_RELATIVE_TO_SECTOR)
        LIQ_adj=np.where(LIQUIDITY_RELATIVE_TO_SECTOR > 0, 0, liq)

    Finance_cost=(df.Finance_cost/(df.Short_term_Liabilities+df.Long_term_Liabilities)+0.1)*-26.95374908
    Asset_turnover_ratio=(df.Revenue/(df.CURRENT_ASSETS+df.Non_Current_Assets)-0.4)*-1.404701266
    return_on_assets=(df.EARNING_BEFORE_TAX/(df.CURRENT_ASSETS+df.Non_Current_Assets))*-8.532932359
    Solvency=((df.Short_term_Liabilities+df.Long_term_Liabilities)/(df.CURRENT_ASSETS+df.Non_Current_Assets)-0.2)*2.902960506
    fin=np.where(Finance_cost<-2.695374908, -2.695374908, Finance_cost)
    FIN_adj=np.where(Finance_cost>0, 0, fin)
    Asset=np.where(Asset_turnover_ratio>0, 0, Asset_turnover_ratio)
    Asset_adj=np.where(Asset_turnover_ratio< -1.68564152, -1.68564152, Asset)
    ret=np.where(return_on_assets>0, 0, return_on_assets)
    Ret_adj=np.where(return_on_assets<-1.10074827, -1.10074827, ret)
    Sol=np.where(Solvency>2.3223684, 2.3223684, Solvency)
    Sol_adj=np.where(Solvency< 0, 0, Sol)
    
    Credit_sc_adj=0.332094193 + sector_growth+sector_evaluation+LIQ_adj+FIN_adj+Asset_adj+Ret_adj+Sol_adj+Com_adj
    Actual_score= (3.838634311-Credit_sc_adj)/(3.838634311-(-7.868071879))*100

    Var = {'Finance_cost': FC_SHARE-(FC_SHARE*(FIN_adj/-2.695374908)),
           'Return_on_Assets': ROA_SHARE-(ROA_SHARE*(Ret_adj/-1.10074827)),
           'Asset_Turnover_Ratio': ATR_SHARE-(ATR_SHARE*(Asset_adj/-1.68564152)),
           'Liquidity_Relative_to_Sector': LRS_SHARE-(LRS_SHARE*(LIQ_adj/LIQUIDITY_RELATIVE_TO_SECTOR_BEST)),
           'Solvency': 0-(SOL_SHARE*(Sol_adj/2.322368))}

    sort_orders = sorted(Var.items(), key=lambda x: x[1], reverse=True)
    #return [print(i[0], i[1].round(2)) for i in sort_orders if i[1]>5] and print('Best score: ', best_score.round(2), 'Actual score: ', Actual_score.round(2))
    return Actual_score.round(2)


List of sectors:    
Agricultura, ganadería, silvicultura y pesca	1       
Industrias extractivas	2       
Industria manufacturera	3    
Suministro de energía eléctrica, gas, vapor y aire acondicionado	4      
Suministro de agua, actividades de saneamiento, gestión de residuos y descontaminación	5      
Construcción	6      
Comercio al por mayor y al por menor y reparación de vehículos de motor y motocicletas	7       
Transporte y almacenamiento	8      
Hostelería	9       
Información y comunicaciones	10       
Actividades financieras y de seguros	11       
Actividades inmobiliarias	12       
Actividades profesionales, científicas y técnicas	13       
Actividades administrativas y servicios auxliares	14      
Administración Pública y defensa y Seguridad Social obligatoria	15     
Educación	16       
Actividades sanitarias y de servicios sociales	17     
Actividades artísticas, recreativas y de entrenimiento	18      
Otros servicios	19       
Actividades de los hogares como empleadores de personal doméstico; actividades de los hogares como productores de bienes y servicios para uso propio	20        
Actividades de organizaciones y organismos extraterritoriales	21         

In [3]:
sector=13 # we need to change sector regarding the sector number assigh above like >>> Industria manufacturera 3
data['Credit_score'] = np.nan
for i in range(0, len(data)):
    data.loc[i, 'Credit_score'] = Credit_Score(data.iloc[i:i+1,], sector)
    
data['Category_type'] = ' '

for i in range(0, len(data)):
    if data.loc[i, 'Credit_score'] < 50:
        data.loc[i, 'Category_type'] = 'Poor'
    elif data.loc[i, 'Credit_score'] < 66:
        data.loc[i, 'Category_type'] = 'Fair'
    elif data.loc[i, 'Credit_score'] < 80:
        data.loc[i, 'Category_type'] = 'Good'
    elif data.loc[i, 'Credit_score'] < 90:
        data.loc[i, 'Category_type'] = 'Very Good'
    else:
        data.loc[i, 'Category_type'] = 'Excellent'

In [4]:
data.to_csv('Extracted_data_with_CS.csv', index=False)